In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import requests
from bs4 import BeautifulSoup
import re
import dateutil
import sys
import logging
from typing import List
from google.cloud import bigquery
from google.cloud import bigquery
from google.oauth2 import service_account

In [10]:
#---------------------Data Cleaning/Transformation-----------------------------------------
#read csv to dataframe
weather_df = pd.read_csv('./data/noaa_weather_data.csv', index_col='DATE')
#rename and filter columns which can be useful
core_weather_df = weather_df[['NAME','PRCP', 'SNOW','SNWD', 'TMAX','TMIN']].copy()
core_weather_df.columns = ['name','precip', 'snow', 'snow_depth', 'temp_max','temp_min']
#fill missing data points
core_weather_df['precip'] = core_weather_df['precip'].fillna(0)
core_weather_df['snow'] = core_weather_df['snow'].fillna(0)
core_weather_df['snow_depth'] = core_weather_df['snow_depth'].fillna(0)
#map the name to a number variable to be used as a predictor
core_weather_df['name_num'] = pd.factorize(core_weather_df['name'])[0]
#'ffill' ('forward fill') fills the value in with the value from the previous date this will apply to the temp_min/max columns since the others have already been filled in
core_weather_df = core_weather_df.fillna(method='ffill')
#change date index to a datetime data type
core_weather_df.index = pd.to_datetime(core_weather_df.index)
#make a visual representation of the data and remove cities that have large chunks of missing data
core_weather_df = core_weather_df[core_weather_df.name.isin(['BEMIDJI, MN US', 'SHARJAH INTER. AIRP, AE']) == False]
# core_weather_df.groupby(by='name').plot()
#create a column 'target_temp_max' by shifting all values in the temp_max column back a day...creates a column based on temperatures from tomorrows temps
core_weather_df['target_temp_max_day_1'] = core_weather_df.groupby('name')['temp_max'].shift(-1)
core_weather_df['target_temp_max_day_2'] = core_weather_df.groupby('name')['target_temp_max_day_1'].shift(-1)
core_weather_df['target_temp_max_day_3'] = core_weather_df.groupby('name')['target_temp_max_day_2'].shift(-1)
core_weather_df['target_temp_max_day_4'] = core_weather_df.groupby('name')['target_temp_max_day_3'].shift(-1)
core_weather_df['target_temp_max_day_5'] = core_weather_df.groupby('name')['target_temp_max_day_4'].shift(-1)
core_weather_df['target_temp_max_day_6'] = core_weather_df.groupby('name')['target_temp_max_day_5'].shift(-1)
core_weather_df['target_temp_max_day_7'] = core_weather_df.groupby('name')['target_temp_max_day_6'].shift(-1)
core_weather_df['target_temp_max_day_8'] = core_weather_df.groupby('name')['target_temp_max_day_7'].shift(-1)
core_weather_df['target_temp_max_day_9'] = core_weather_df.groupby('name')['target_temp_max_day_8'].shift(-1)
core_weather_df['target_temp_max_day_10'] = core_weather_df.groupby('name')['target_temp_max_day_9'].shift(-1)
#remove the last row since its a NaN value cuz that value would be in the future
core_weather_df = core_weather_df.dropna()
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
#test set includes all data after and including jan 1st 2022
test_set = core_weather_df.loc['2022-01-01':]
#adding todays temp to test set
# test_set_today = pd.DataFrame({
#     'name' : ['PORTLAND INTERNATIONAL AIRPORT, OR US'],
#     "precip": [0],
#     "snow": [0],
#     "snow_depth": [0],
#     "temp_max": [45],
#     "temp_min": [33]
# }, index=["2023-03-03"])

# test_set_today.index = pd.to_datetime(test_set_today.index)

# test_set  = pd.concat([test_set, test_set_today])

In [11]:
#------------scrape current data an append to historical dataset---------------------------------
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail (Humidity, Wind Speed, Barometer, Dewpoint, Visibility, Last update)
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    station = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['weather_station'] = station

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station
0,44°F,Low: 33 °F,High: 42 °F,29.94 in (1013.89 mb),36°F (2°C),72%,03 Mar 03:21 PM PST,7.00 mi,38°F (3°C),SW 13 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
1,37°F,Low: 24 °F,High: 34 °F,29.43 in (996.6 mb),24°F (-4°C),59%,3 Mar 1:53 pm AKST,10.00 mi,31°F (-1°C),E 8 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [12]:
urls = ['https://www.localconditions.com/weather-portland-oregon/97201/past.php',
        "https://www.localconditions.com/weather-sitka-alaska/99835/past.php",
        ]

precip_df = pd.DataFrame()

for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        details = soup.select_one(".past_weather_express")
        # Find all div elements with class="panel"
        panel_divs = soup.find_all('div', {'class': 'panel'})
        # Extract the text content of each div element and store it in a list
        panel_texts = [panel_div.text.strip() for panel_div in panel_divs]
        # Print the list of extracted text content
        data = panel_texts[1]
        data = [item.strip() for item in data]
        data = [item for item in data if item]
        data = data[79:90]
        data="".join(data)
        df = pd.DataFrame([data], columns=['precip'])
        precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)
        
precip_df['precip'] = precip_df['precip'].str.extract(pat='(\d+\.?\d*)').astype(float)
precip_df = precip_df.fillna(0)

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df)

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,precip
0,44°F,Low: 33 °F,High: 42 °F,29.94 in (1013.89 mb),36°F (2°C),72%,03 Mar 03:21 PM PST,7.00 mi,38°F (3°C),SW 13 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.039
1,37°F,Low: 24 °F,High: 34 °F,29.43 in (996.6 mb),24°F (-4°C),59%,3 Mar 1:53 pm AKST,10.00 mi,31°F (-1°C),E 8 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.037


In [13]:
#-----------Data Transformations-----------------

# Convert 'lat' and 'lon' columns to float type
source_df['latitude'] = source_df['latitude'].astype(float)
source_df['longitude'] = source_df['longitude'].astype(float)

# Convert 'elev' column to int type
source_df['elevation_ft'] = source_df['elevation_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
source_df['temp'] = source_df['temp'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the tempmin string and convert it to int
source_df['tempmin'] = source_df['tempmin'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the temperature string and convert it to int
source_df['tempmax'] = source_df['tempmax'].str.extract('(\d+)').astype(float)

# Split wind speed values into components and convert speed to int type
source_df['Wind Speed'] = source_df['Wind Speed'].str.extract('(\d+)', expand=False).fillna(0).astype(float)

# Convert 'humidity' column to int type
source_df['Humidity'] = source_df['Humidity'].str.extract('(\d+)', expand=False).astype(float)

# Convert 'barometer' column to float type, and convert inches to millibars
source_df['Barometer'] = round(source_df['Barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None), 2)

# Convert 'Visibility' column to float type
source_df['Visibility'] = source_df['Visibility'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

#Convert 'last_update' column to UTC
source_df['Last update'] = source_df['Last update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={"EST": -5 * 3600, "CST": -6 * 3600, "MST": -7 * 3600,"PST": -8 * 3600,"AKST": -9 * 3600,"HST": -10 * 3600}))
source_df['Last update'] = source_df['Last update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))
source_df['datetime'] = source_df['Last update'].dt.strftime('%Y-%m-%d')
source_df['datetime'] = pd.to_datetime(source_df['datetime'])

# make wind chill a float if exists and only display degree F
try:
    source_df[['Wind Chill']] = source_df['Wind Chill'].str.extract('(\d+)', expand=True).astype(float)
except:
    None

# extract the numeric value of dewpoint and only display the degree n farenheit
source_df[['Dewpoint']] = source_df['Dewpoint'].str.extract('(\d+)', expand=True).astype(float)

#change precip data type to float
source_df['precip'] = source_df['precip'].astype(float)

#rename weather station column to the city
def rename_station(value):
    if value == 'Portland, Portland International Airport (KPDX)':
        return 'PORTLAND INTERNATIONAL AIRPORT, OR US'
    elif value == 'Sitka - Sitka Airport (PASI)':
        return 'SITKA AIRPORT, AK US'

source_df['name'] = source_df['weather_station'].map(rename_station)

#change the names and order of columns to better fit the historical data
source_df = source_df.rename({'Humidity': 'humidity', 'Wind Speed': 'windspeed', 'Visibility': 'visibility','Wind Chill': 'windchill','Dewpoint':'dewpoint'}, axis=1) 
#this line only includes necesarry columns
source_df = source_df.reindex(['name','datetime','tempmax','tempmin','temp','windchill','dewpoint','humidity','precip','windspeed','visibility'], axis=1)
source_df = source_df.fillna(0)
source_df = source_df.set_index(['datetime'])

test_set = pd.concat([test_set, source_df])
display(test_set)

,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,...,target_temp_max_day_9,target_temp_max_day_10,tempmax,tempmin,temp,windchill,dewpoint,humidity,windspeed,visibility
2022-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.000,0.1,33.9,-7.0,-17.0,0.0,-16.0,-14.0,-19.0,...,3.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.000,0.0,33.1,-16.0,-47.0,0.0,-14.0,-19.0,-26.0,...,7.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.000,0.0,33.1,-14.0,-45.0,0.0,-19.0,-26.0,-6.0,...,12.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.000,0.0,33.1,-19.0,-37.0,0.0,-26.0,-6.0,-26.0,...,21.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.000,0.0,33.1,-26.0,-38.0,0.0,-6.0,-26.0,-30.0,...,20.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-12,"ORLANDO EXECUTIVE AIRPORT, FL US",0.020,0.0,0.0,72.0,54.0,12.0,71.0,77.0,82.0,...,85.0,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-13,"ORLANDO EXECUTIVE AIRPORT, FL US",0.000,0.0,0.0,71.0,49.0,12.0,77.0,82.0,85.0,...,87.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-14,"ORLANDO EXECUTIVE AIRPORT, FL US",0.000,0.0,0.0,77.0,48.0,12.0,82.0,85.0,87.0,...,88.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-03,Portland,0.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,42.0,33.0,44.0,38.0,36.0,72.0,13.0,7.0


In [14]:
#-----------------create/test data model--------------------------------
#using Ridge regression to minimize overfitting
regression = Ridge(alpha=.1)
predictors = ['precip','snow','snow_depth','temp_max','temp_min']
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
#test set includes all data after and including jan 1st 2022
test_set = core_weather_df.loc['2022-01-01':]

# test_set_shifted = test_set.shift(365, freq='D')
# test_set2= test_set_shifted.loc['2023-01-01':'2023-12-31']

#adding todays temp to test set
test_set_today = pd.DataFrame({
    'name' : ['PORTLAND INTERNATIONAL AIRPORT, OR US'],
    "precip": [0],
    "snow": [0],
    "snow_depth": [0],
    "temp_max": [45],
    "temp_min": [33]
}, index=["2023-03-03"])

test_set_today.index = pd.to_datetime(test_set_today.index)

test_set  = pd.concat([test_set, test_set_today])
#train the model based on the predictors
regression.fit(training_set[predictors], training_set['target_temp_max_day_1'])
predictions = regression.predict(test_set[predictors])

combined_df = pd.concat([test_set[['name']], pd.Series(predictions, index = test_set.index)], axis=1)
combined_df.columns = ['name','Predicted_temp_next_day']

core_weather_df.name.unique()
display(combined_df.loc[combined_df['name'] == 'PORTLAND INTERNATIONAL AIRPORT, OR US'])
display(test_set.tail())


,name,Predicted_temp_next_day
2022-01-01,"PORTLAND INTERNATIONAL AIRPORT, OR US",36.544817
2022-01-02,"PORTLAND INTERNATIONAL AIRPORT, OR US",49.337245
2022-01-03,"PORTLAND INTERNATIONAL AIRPORT, OR US",48.294390
2022-01-04,"PORTLAND INTERNATIONAL AIRPORT, OR US",48.031114
2022-01-05,"PORTLAND INTERNATIONAL AIRPORT, OR US",43.994891
...,...,...
2023-02-11,"PORTLAND INTERNATIONAL AIRPORT, OR US",52.957451
2023-02-12,"PORTLAND INTERNATIONAL AIRPORT, OR US",51.579057
2023-02-13,"PORTLAND INTERNATIONAL AIRPORT, OR US",49.353518
2023-02-14,"PORTLAND INTERNATIONAL AIRPORT, OR US",46.550784


,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
2023-02-11,"ORLANDO EXECUTIVE AIRPORT, FL US",0.01,0.0,0.0,84.0,67.0,12.0,72.0,71.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0
2023-02-12,"ORLANDO EXECUTIVE AIRPORT, FL US",0.02,0.0,0.0,72.0,54.0,12.0,71.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0,87.0
2023-02-13,"ORLANDO EXECUTIVE AIRPORT, FL US",0.00,0.0,0.0,71.0,49.0,12.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0,87.0,88.0
2023-02-14,"ORLANDO EXECUTIVE AIRPORT, FL US",0.00,0.0,0.0,77.0,48.0,12.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0,87.0,88.0,88.0
2023-03-03,"PORTLAND INTERNATIONAL AIRPORT, OR US",0.00,0.0,0.0,45.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
test_set2 = test_set.shift(365, freq='D')
display(test_set.head())
display(test_set2.head())

,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
2022-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.1,33.9,-7.0,-17.0,0.0,-16.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0
2022-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-16.0,-47.0,0.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0
2022-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-14.0,-45.0,0.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0
2022-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-19.0,-37.0,0.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0
2022-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-26.0,-38.0,0.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0,7.0


,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
2023-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.1,33.9,-7.0,-17.0,0.0,-16.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0
2023-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-16.0,-47.0,0.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0
2023-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-14.0,-45.0,0.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0
2023-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-19.0,-37.0,0.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0
2023-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-26.0,-38.0,0.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0,7.0
